数据加载、存储与文件格式

In [1]:
import pandas as pd
import numpy as np

# 文本文件

## 读取文本格式的数据

pandas中的解析函数

| 函数 | 说明 |
|------|------|
|`read_csv`|从文件、URL、文件型对象加载带分隔符的数据，默认分隔符为逗号|
|`read_table`|从文件、URL、文件型对象加载带分隔符的数据。默认分隔符为制表符"\t"|
|`read_fwf`|读取定宽列格式数据，即：无分隔符|
|`read_clipboard`|读取粘贴板数据|

In [2]:
!cat './examples/ex1.csv'

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
df = pd.read_csv('./examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_table('./examples/ex1.csv', sep=',')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
!cat './examples/ex2.csv'

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [6]:
# 数据无标题行
df = pd.read_csv('./examples/ex2.csv', header=None)
df

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# 指定列索引
df = pd.read_csv('./examples/ex2.csv',names=['a', 'b', 'c', 'd', 'message'])
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# 指定行索引
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [9]:
pd.read_csv('./examples/ex2.csv', names=names, index_col=4)

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
!cat './examples/csv_mindex.csv'

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [11]:
parsed = pd.read_csv('./examples/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
!cat './examples/ex3.txt'

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [13]:
result = pd.read_table('./examples/ex3.txt', sep='\s+') # 空格分隔
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


+ 默认第一行为标题行，列名比标题行少1，故推断第一列是DataFrame的行索引。

In [14]:
!cat './examples/ex4.csv'

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [15]:
df = pd.read_csv('./examples/ex4.csv', skiprows=[0,2,3])
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 默认情况下，pandas会用一组经常出现的标记值进行识别，NA、-1、#IND、NULL等。
+ 也可以用`na_values`指定一组表示缺失值的字符串。

In [16]:
!cat './examples/ex5.csv'

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [17]:
result=pd.read_csv("./examples/ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
result.isnull()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [19]:
result = pd.read_csv("./examples/ex5.csv", na_values=['NULL',])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


+ 也可以用字典为各列指定不同的NA标记值。

In [20]:
!cat './examples/ex5.csv'
sentinels = {'message':['foo', 'NA'], 'something':['two']}
pd.read_csv('./examples/ex5.csv', na_values=sentinels)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


`read_csv`和`read_table`函数的参数

| 参数 |  说明 |
|----|-----|
|`path`|文件系统位置、URL、文件型对象的字符串|
|`sep`或`delimiter`|用于对列中各字段进行拆分的字符序列或正则表达式|
|`header`|用作列名的**行号**，默认为0，如果没有header行则应设置`header=None`|
|`index_col`|用作行索引的列编号或列名。可以是单个名称/数字，或由多个名称/数字组成的列表（层次化索引）|
|`names`|用于结果的列名列表，配合`header=None`|
|`skiprows`|需要跳过的行数（从文件开始算起），或需要跳过行的行号列表（从0开始）|
|`na_values`|一组用于替换为NaN的值|
|`parse_dates`|尝试将数据解析为日期，默认为False。如果为True，则尝试解析所有列。也可以指定需要解析的一组列号或列名。如果列表的元素为列表或元组，则会将多个列组合到一起在进行日期解析工作。|
|`encoing`|用于unicode的文本编码格式。|
|`squeeze`|如果数据解析后只有一列，则返回Series|

+ 读取大文件时，读取小部分或逐块对文件进行迭代

In [21]:
result = pd.read_csv("./examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [22]:
pd.read_csv('./examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


## 将数据写出到文本格式
用用DataFrame的`to_csv`函数

In [23]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [24]:
data.to_csv('examples/out.csv')   # 写入到以逗号分割的文件中
!cat './examples/out.csv'

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [25]:
data.to_csv('examples/out2.csv', sep='|')
!cat './examples/out2.csv'

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


+ 缺失值在输出结果中默认会被表示为空字符串，也可以通过`na_rep`参数表示为别的标记值。

In [26]:
import sys
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


+ 默认会输出行和列的标签，但也可以用`index=False`和`header=False`分别禁用行索引和列索引。
+ 还可以只写出部分列，并`columns`指定顺序。

In [27]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [28]:
data.to_csv(sys.stdout, index=False, 
            columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [29]:
dates = pd.date_range('1/1/2018', periods=7)
dates
ts = pd.Series(np.arange(7), index=dates)
ts

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07'],
              dtype='datetime64[ns]', freq='D')

2018-01-01    0
2018-01-02    1
2018-01-03    2
2018-01-04    3
2018-01-05    4
2018-01-06    5
2018-01-07    6
Freq: D, dtype: int32

In [30]:
ts.to_csv('examples/ts.csv') # Series的输出

In [31]:
!cat './examples/ts.csv'

,0
2018-01-01,0
2018-01-02,1
2018-01-03,2
2018-01-04,3
2018-01-05,4
2018-01-06,5
2018-01-07,6


In [32]:
pd.read_csv('./examples/ts.csv', parse_dates=True, header=0, index_col=0)

,0
2018-01-01,0
2018-01-02,1
2018-01-03,2
2018-01-04,3
2018-01-05,4
2018-01-06,5
2018-01-07,6


## 手工处理分隔符

In [33]:
!cat './examples/ex7.csv'

"a","b","c"
"1","2","3"
"4","5","6","7"


In [34]:
# 由于文件行长度不一致，本命令会出错
pd.read_csv('./examples/ex7.csv')

ParserError: Error tokenizing data. C error: Expected 3 fields in line 3, saw 4


In [35]:
import csv
f = open('examples/ex7.csv')  # 打开文件
reader = csv.reader(f)  # 读取文件
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['4', '5', '6', '7']


In [36]:
lines = list(csv.reader(open('./examples/ex7.csv')))
lines
header, values = lines[0], lines[1:]

[['a', 'b', 'c'], ['1', '2', '3'], ['4', '5', '6', '7']]

In [37]:
header

['a', 'b', 'c']

In [38]:
values

[['1', '2', '3'], ['4', '5', '6', '7']]

In [40]:
list(zip(values[0],values[1]))

[('1', '4'), ('2', '5'), ('3', '6')]

In [41]:
list(zip(*values))

[('1', '4'), ('2', '5'), ('3', '6')]

 `*` operator to unpack the arguments out of a list or tuple.For examle:

`list(range(3, 6))            # normal call with separate arguments
[3, 4, 5]
args = [3, 6]
list(range(*args))            # call with arguments unpacked from a list
[3, 4, 5]`

In [42]:
for z in zip(*values):
    print(z)

('1', '4')
('2', '5')
('3', '6')


In [43]:
data_list = [(h,v) for h,v in zip(header, zip(*values))]
data_list

[('a', ('1', '4')), ('b', ('2', '5')), ('c', ('3', '6'))]

In [44]:
data_dict = {h:v for h,v in zip(header,zip(*values))}
data_dict

{'a': ('1', '4'), 'b': ('2', '5'), 'c': ('3', '6')}

In [45]:
df = pd.DataFrame(data_dict)
df

,a,b,c
0,1,2,3
1,4,5,6


# JSON数据
+ JSON（JavaScript Object Notation）,通过HTTP请求在web浏览器和其他应用程序直接按发送数据的标准格式之一。比表格型文本格式（如CSV）更灵活。
+ Python标准库中内置了`json`库，通过`json.loads`可以将JSON字符串转换成Python形式。

## Python标准库读取

```JSON
obj = """
{"name":"Wes",
"places_lived":["United States", "Spain", "Germany"],
"pet":null,
"siblings":[{"names":"Scott", "age":25, "pet":"Zuko"},{"name":"Katie", "age":33, "pet":"Cisco"}]
}"""
```

In [46]:
import json

In [47]:
obj = """
{"name":"Wes",
"places_lived":["United States", "Spain", "Germany"],
"pet":null,
"siblings":[{"name":"Scott", "age":25, "pet":"Zuko"},{"name":"Katie", "age":33, "pet":"Cisco"}]
}"""

In [48]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 25, 'pet': 'Zuko'},
  {'name': 'Katie', 'age': 33, 'pet': 'Cisco'}]}

In [49]:
result['name']

'Wes'

In [50]:
result['siblings']

[{'name': 'Scott', 'age': 25, 'pet': 'Zuko'},
 {'name': 'Katie', 'age': 33, 'pet': 'Cisco'}]

In [51]:
siblings = pd.DataFrame(result['siblings'])
siblings

,name,age,pet
0,Scott,25,Zuko
1,Katie,33,Cisco


## Pandas读取

+ pandas 已有`read_json`和`to_json`函数，具体看查阅相关函数帮助。其主要参数`orient`可用于指定读入数据的格式：

|参数|说明|
|----|----|
|`split` |dict like {index -> [index], columns -> [columns], data -> [values]}|
|`records`|list like [{column -> value}, ... , {column -> value}]|
|`index`|dict like {index -> {column -> value}}|
|`columns` |dict like {column -> {index -> value}}|
|`values` |just the values array|

In [52]:
df = pd.DataFrame([['a', 'b'], ['c', 'd']],
                  index=['row 1', 'row 2'],
                  columns=['col 1', 'col 2'])
df

,col 1,col 2
row 1,a,b
row 2,c,d


In [53]:
df.to_json('./examples/json_split', orient="split")

In [54]:
!cat ./examples/json_split

{"columns":["col 1","col 2"],"index":["row 1","row 2"],"data":[["a","b"],["c","d"]]}


In [55]:
df.to_json(orient='records')

'[{"col 1":"a","col 2":"b"},{"col 1":"c","col 2":"d"}]'

In [56]:
df.to_json(orient='index')

'{"row 1":{"col 1":"a","col 2":"b"},"row 2":{"col 1":"c","col 2":"d"}}'

In [57]:
df.to_json(orient='columns')

'{"col 1":{"row 1":"a","row 2":"c"},"col 2":{"row 1":"b","row 2":"d"}}'

In [58]:
df.to_json(orient='values')

'[["a","b"],["c","d"]]'

In [59]:
!cat './examples/test.json'

{"columns":[0,1,2],"index":["a","b","c"],"data":[[1,4,7],[2,5,8],[3,6,9]]}


In [60]:
df = pd.read_json('./examples/test.json', orient='split')
df

,0,1,2
a,1,4,7
b,2,5,8
c,3,6,9


# Excel数据

## 读取Excle数据
+ pandas读取Excel文件用到了`xlrd`和`openpyxl`包，因此必须事先安装。
+ pandas的`read_excel`函数读取文件返回DataFrame


In [61]:
pd.read_excel('./examples/ex1.xlsx')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [62]:
df = pd.read_excel('./examples/ch7.xlsx', sheet_name=1)
df

,teacher_id,teacher_name
0,1,zhangsan
1,2,lisi
2,3,wangwu
3,4,zhaoliu


+ 也可以直接用`xlrd`包读取excel文件。

In [63]:
import xlrd

In [64]:
book = xlrd.open_workbook('./examples/projectdata.xlsx')
for i in range(book.nsheets):
    table = book.sheets()[i]
    data = [table.cell(row, col).value for (row, col) in ((2,2),(2,6),(4,2))]
    print(data)

['野店村', '野店镇', 2195.0]
['焦坡村', '野店镇', 1497.0]
['东山村', '野店镇', 623.0]
['朱家坡', '野店镇', 1150.0]
['苏家沟', '野店镇', 444.0]
['烟庄', '野店镇', 1081.0]
['毛坪村', '野店镇', 1365.0]
['伊家圈', '野店镇', 809.0]
['石泉', '野店镇', 1638.0]
['新庄村', '野店镇', 1100.0]
['南峪村', '野店镇', 994.0]
['南晏子', '野店镇', 994.0]
['北坪', '野店镇', 955.0]
['棋盘石', '野店镇', 1743.0]
['安平崮', '野店镇', 1601.0]
['东坪', '野店镇', 1357.0]
['上东门', '野店镇', 1791.0]
['板崮崖', '野店镇', 1975.0]
['桑子峪', '野店镇', 1300.0]
['梭庄', '野店镇', 1013.0]
['石槽', '野店镇', 895.0]
['黄崖', '野店镇', 791.0]
['大山', '野店镇', 1117.0]
['大石头', '野店镇', 1027.0]
['黄土良', '野店镇', 854.0]
['演马庄', '野店镇', 1186.0]
['北晏子', '野店镇', 766.0]
['朝阳', '野店镇', 473.0]
['寨后万', '野店镇', 468.0]
['立新', '野店镇', 521.0]
['白杨峪', '野店镇', 666.0]
['郭庄', '野店镇', 732.0]
['翻金峪', '野店镇', 539.0]
['西梭庄', '野店镇', 288.0]
['马头崖', '野店镇', 351.0]
['对景峪', '野店镇', 253.0]
['石仁坡', '野店镇', 275.0]


## Excel文件写入

In [65]:
df1 = pd.DataFrame([['a', 'b'], ['c', 'd']],
                   index=['row 1', 'row 2'],
                   columns=['col 1', 'col 2']) 
df1

,col 1,col 2
row 1,a,b
row 2,c,d


In [66]:
df1.to_excel("./examples/output.xlsx") 

In [67]:
df2 = df1.copy()
with pd.ExcelWriter('./examples/output2.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Sheet_name_1')
    df2.to_excel(writer, sheet_name='Sheet_name_2')

# 使用HTML和Web API获取数据
许多网站提供通过JSON或其他格式提供数据的公共API，通过Python访问API，譬如`request`包

In [68]:
import requests
import json

In [69]:
url = 'http://api.map.baidu.com/geocoder?'
params = {}
params['output'] = 'json'
params['address'] = '上海海事大学'
params['ak'] = 'WS4aniHIt4ITPIVnbwfvN54DynG00a7T'
params

{'output': 'json',
 'address': '上海海事大学',
 'ak': 'WS4aniHIt4ITPIVnbwfvN54DynG00a7T'}

In [73]:
resp = requests.get(url, params=params)

In [74]:
resp

<Response [200]>

In [75]:
data = json.loads(resp.text)
data

{'status': 'OK',
 'result': {'location': {'lng': 116.413384, 'lat': 39.910925},
  'precise': 0,
  'confidence': 20,
  'level': '城市'}}

In [76]:
df = pd.DataFrame(data['result']['location'], index=['SMU'])
df

,lng,lat
SMU,116.413384,39.910925


# 使用数据库
将数据从SQL加载到DataFrame

In [77]:
import sqlite3

In [78]:
conn = sqlite3.connect('./examples/data.sqlite')

In [79]:
cursor = conn.execute("select Name, Town, Pop, Lat, Lng from village")
rows = cursor.fetchall()
rows

[('下东门村', '坦埠镇', 2045, 35.852457, 118.189404),
 ('东坦埠村', '坦埠镇', 1853, 35.801192, 118.228538),
 ('东崖村', '坦埠镇', 1302, 35.824206, 118.215981),
 ('东河南村', '坦埠镇', 1063, 35.789275, 118.234346),
 ('东西崖村', '坦埠镇', 2120, 35.841863, 118.202611),
 ('代家庄村', '坦埠镇', 1181, 35.74744, 118.036742),
 ('南极山村', '坦埠镇', 325, 35.843734, 118.187794),
 ('响水庄村', '坦埠镇', 550, 35.812858, 118.206505),
 ('官庄村', '坦埠镇', 1220, 35.74744, 118.036742),
 ('张家庄村', '坦埠镇', 1768, 35.822732, 118.13797),
 ('故县村', '坦埠镇', 1859, 35.802529, 118.207691),
 ('来石万村', '坦埠镇', 515, 35.812489, 118.194266),
 ('来石庄村', '坦埠镇', 701, 35.806575, 118.20942),
 ('水明崖村', '坦埠镇', 1060, 35.789972, 118.181779),
 ('沙沟村', '坦埠镇', 1062, 35.784318, 118.207658),
 ('海龙万村', '坦埠镇', 329, 35.820998, 118.166729),
 ('潘庄村', '坦埠镇', 1169, 35.78692, 118.195158),
 ('砚池万村', '坦埠镇', 518, 35.829046, 118.180634),
 ('西坦埠村', '坦埠镇', 1518, 35.80605, 118.227207),
 ('西崖村', '坦埠镇', 1337, 35.836513, 118.198651),
 ('西河南村', '坦埠镇', 1142, 35.789267, 118.222858),
 ('诸夏村', '坦埠镇', 2045, 35.786453

In [80]:
# This read-only property returns a list of tuples describing the columns in a result set. 
cursor.description

(('Name', None, None, None, None, None, None),
 ('Town', None, None, None, None, None, None),
 ('Pop', None, None, None, None, None, None),
 ('Lat', None, None, None, None, None, None),
 ('Lng', None, None, None, None, None, None))

In [81]:
list(zip(*cursor.description))[0]

('Name', 'Town', 'Pop', 'Lat', 'Lng')

In [82]:
pd.DataFrame(rows, columns=list(zip(*cursor.description))[0])

,Name,Town,Pop,Lat,Lng
0,下东门村,坦埠镇,2045,35.852457,118.189404
1,东坦埠村,坦埠镇,1853,35.801192,118.228538
2,东崖村,坦埠镇,1302,35.824206,118.215981
3,东河南村,坦埠镇,1063,35.789275,118.234346
4,东西崖村,坦埠镇,2120,35.841863,118.202611
...,...,...,...,...,...
241,翻金峪,野店镇,539,35.956199,118.084083
242,西梭庄,野店镇,288,35.934126,118.088294
243,马头崖,野店镇,351,35.936568,118.099446
244,对景峪,野店镇,253,35.965390,118.056732
